In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(9006, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 0 0 

## run

In [8]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   619   0  0    986   1243  1031  0.03  0.37  0.45  0.80  0.80
1   627   1  0    962   1219  1007  0.04  0.41  0.45  0.84  0.83
2   691   2  0    939   1194   984  0.04  0.43  0.45  0.86  0.85
3   792   3  0    919   1169   964  0.04  0.44  0.45  0.86  0.86
4   769   4  0    900   1150   945  0.05  0.45  0.45  0.85  0.84
5   813   5  0    888   1137   935  0.05  0.46  0.45  0.83  0.83
6   829   6  0    878   1127   926  0.05  0.46  0.45  0.85  0.84
7   827   7  0    872   1119   920  0.06  0.46  0.45  0.84  0.84
8   866   8  0    866   1114   915  0.06  0.45  0.45  0.86  0.86
9   892   9  0    861   1110   910  0.06  0.45  0.45  0.85  0.85
10  883  10  0    856   1105   906  0.06  0.45  0.45  0.85  0.85
11  884  11  0    853   1101   902  0.06  0.45  0.45  0.85  0.85
12  800  12  0    850   1098   899  0.06  0.45  0.45  0.84  0.84
13  882  13  0    847   1095   898  0.07  0.44  0.45  0.82  0.82
14  760  14  0    845   1093   895  0.07  0.44  0.45  0.82  0.82
15  791  15  0    844   1091   894  0.07  0.44  0.45  0.82  0.81
16  700  16  0    844   1091   893  0.07  0.44  0.45  0.82  0.81
17  775  17  0    842   1088   891  0.07  0.44  0.45  0.81  0.81
18  796  18  0    841   1088   890  0.07  0.44  0.45  0.80  0.80
19  766  19  0    840   1086   889  0.07  0.44  0.45  0.79  0.79
20  809  20  0    838   1082   887  0.07  0.44  0.45  0.79  0.79
21  713  21  0    837   1079   886  0.08  0.44  0.45  0.79  0.78
22  819  22  0    836   1077   884  0.08  0.44  0.45  0.78  0.78
23  789  23  0    837   1075   884  0.08  0.43  0.45  0.79  0.78
24  752  24  0    836   1073   884  0.08  0.43  0.45  0.79  0.78
25  829  25  0    835   1073   882  0.08  0.43  0.45  0.78  0.78
26  745  26  0    834   1071   881  0.09  0.43  0.45  0.78  0.78
27  794  27  0    834   1070   881  0.09  0.43  0.45  0.78  0.78
28  733  28  0    833   1067   879  0.09  0.43  0.45  0.78  0.78
29  696  29  0    831   1064   877  0.09  0.43  0.45  0.78  0.78
30  761  30  0    830   1062   877  0.10  0.43  0.45  0.78  0.77
31  721  31  0    829   1060   875  0.10  0.42  0.45  0.78  0.77
32  817  32  0    828   1056   873  0.10  0.42  0.45  0.78  0.77
33  848  33  0    826   1052   872  0.11  0.42  0.45  0.78  0.77
34  714  34  0    826   1049   870  0.11  0.42  0.45  0.78  0.77
35  803  35  0    824   1048   868  0.11  0.42  0.45  0.78  0.77
36  796  36  0    824   1046   868  0.12  0.42  0.45  0.78  0.77
37  789  37  0    823   1045   867  0.12  0.42  0.45  0.78  0.77
38  535  38  0    822   1046   867  0.12  0.42  0.45  0.78  0.77
39  522  39  0    822   1047   867  0.13  0.41  0.45  0.78  0.77
40  472  40  0    823   1047   867  0.13  0.41  0.45  0.78  0.77
41  520  41  0    823   1047   867  0.13  0.41  0.45  0.78  0.77
42  537  42  0    824   1047   867  0.13  0.41  0.45  0.78  0.77
43  497  43  0    823   1046   867  0.13  0.41  0.45  0.78  0.77
44  488  44  0    823   1046   867  0.13  0.42  0.45  0.81  0.80
45  478  45  0    823   1047   867  0.13  0.41  0.45  0.78  0.77
46  541  46  0    824   1047   867  0.13  0.41  0.45  0.78  0.77
47  474  47  0    823   1048   867  0.13  0.41  0.45  0.78  0.77
48  435  48  0    824   1047   867  0.14  0.41  0.45  0.78  0.77
49  423  49  0    824   1048   867  0.13  0.41  0.45  0.78  0.77

 0 9006 280930.0 use get write one know work like file drive problem
   0-1 5314 258131.0 write one article get say think go make like know
     0-1-1 2085 8753.0 islam argument absolute truth muslim islamic conclusion sex god muslims
     0-1-2 3229 2054.0 detector radar edge gun gas ticket double auto manual illegal
   0-2 189 16160.0 pt game team new vs la period play hockey van
     0-2-1 189 128.0 street michael title plug june roger specifically round feel threat
   0-3 417 9700.0 entry file output program int char printf section oname rule
     0-3-1 417 145.0 warning nt update print iii macintosh matthew modem scale graphic
   0-4 2839 109724.0 key use space system encryption chip one government information people
     0-4-1 2671 2101.0 universe detector physical theory star radar vote militia mass db
     0-4-2 168 63.0 space instruction roger long jesus nature state test crime domain
   0-5 208 5239.0 ground wire outlet wiring circuit neutral water connect jumper box
     0-5